In [ ]:
import torch
import clip
from PIL import Image
from utils import *
from tqdm import tqdm
from dataloader_comma import *
import json
from torch.utils.data import DataLoader
import pandas as pd
import torchvision.utils as vutils
import random
from torchvision import datasets, transforms
import os 
import matplotlib.pyplot as plt

In [ ]:
#!pip install git+https://github.com/openai/CLIP.git
#!pip install h5py

In [ ]:
import torch
from utils import *
import pandas as pd
from tqdm import tqdm
from dataloader_comma import *
import torchvision.utils as vutils
import random
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os 
from utils import get_scenarios

device = "cuda:0"
model, preprocess = clip.load("ViT-L/14", device=device)
multitask = "distance"
dataset_type = 'train'
dataset = CommaDataset(dataset_type=dataset_type, multitask=multitask, return_full=True)
loader = DataLoader(dataset, batch_size=1, num_workers=5)

##CHANGE PATH
save_path = '/kaggle/working/'
dfs = []
scenarios, scenarios_tokens = get_scenarios("retinanet")   #or clip
scenarios = list(scenarios)
text = clip.tokenize(scenarios).to(device)
scenarios_tokens = scenarios_tokens.to(device)
p= '/kaggle/input/scenarios/'

feats = []
similarities = []
similarities_labels = []
with torch.no_grad():
    text_features = model.encode_text(scenarios_tokens)
for elem in os.listdir(p): 
    if ".txt" in elem: continue
    # Open the image using PIL
    img = Image.open(p + elem)
    img = torch.from_numpy(np.array(img)).permute(2, 0, 1).float() / 255.0
    img = img.to(device).unsqueeze(0)
    with torch.no_grad():
        image_features = model.encode_image(img)
    
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    feats.append(image_features)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    with open((p + elem).replace(".png", ".txt"), 'r') as f:
        # Read the contents of the file as a string
        dict_str = f.read()
    similarities_labels.append(list(eval(dict_str).values()))
    similarities.append(similarity)

In [ ]:
feats = torch.concatenate(feats)
stacked = torch.stack([torch.tensor(list(similarities_labels)) for elem in range(240)])


In [ ]:
def get_similarity(image_array, feats, text_features, stacked, intervention):
    with torch.no_grad():
        image_features = model.encode_image(image_array[intervention.squeeze().tolist()])
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    img_similarity = (100.0 * image_features @ feats.T).softmax(dim=-1)
    text_similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    img_sim, max_indices = torch.max(img_similarity, dim=1)
    s = stacked[intervention.squeeze().tolist()]
    selected_indices = s[torch.arange(len(s)), max_indices.cpu()].squeeze(dim=1)
    similarities = (1-img_sim).unsqueeze(dim=1).cpu() * text_similarity.cpu() + (img_sim).unsqueeze(dim=1).cpu() * selected_indices.cpu()
    return similarities

In [ ]:
top_5_reinstate = []
top_5_intervene_before = []
top_5_intervene_after = []

with torch.no_grad():
    for j, batch in tqdm(enumerate(loader)):
        image_array, vego, angle, distance, gas, brake, ccenabled = batch
        image_array = image_array.squeeze().to(device)
        intervention = ccenabled.squeeze()

        #switch_indices = np.where(np.diff(intervention) == 1)[0] + 1
        intervention_after = get_reduced_sample(ccenabled, reinstate=False, after=True)
        intervention_before = get_reduced_sample(ccenabled, reinstate=False, after=False)
        
        if intervention_before.sum() != 0: 
            img_sim_intervention_before = get_similarity(image_array, feats, text_features, stacked, intervention_before)
            values, indices = img_sim_intervention_before.float().topk(5, dim=1)
            indices, idx = torch.sort(indices,dim=1)
            values, indices = indices.topk(1, dim=0)
            top_5_intervene_before.extend(values.tolist())

            img_sim_intervention_after = get_similarity(image_array, feats, text_features, stacked, intervention_after)
            values, indices = img_sim_intervention_after.float().topk(5, dim=1)
            indices, idx = torch.sort(indices,dim=1)
            values, indices = indices.topk(1, dim=0)
            top_5_intervene_after.extend(values.tolist())

        '''
        #reinstate = get_reduced_sample(ccenabled, reinstate=True)
        # if reinstate.sum() != 0: 
            img_sim_reinstate = get_similarity(image_array, feats, text_features, stacked, reinstate)
            values, indices = img_sim_reinstate.float().topk(5,dim=1)
            top_5_reinstate.extend(indices.tolist())
        #values, indices = similarity[0].topk(5)
        
        
        # Print the result'''
        '''print("\nTop predictions:\n")
        for value, index in zip(values, indices):
            print(f"{scenarios[index]:>16s}: {100 * value.item():.2f}%")
        break'''

In [ ]:
top_5_intervene_after_sq = torch.tensor(top_5_intervene_after).reshape((len(top_5_intervene_after)* len(top_5_intervene_after[1]), 1)).squeeze().tolist()

In [ ]:
top_5_intervene_before_sq = torch.tensor(top_5_intervene_before).reshape((len(top_5_intervene_before)* len(top_5_intervene_before[1]), 1)).squeeze().tolist()

In [ ]:
df = pd.DataFrame(top_5_intervene_after_sq, columns=["scenario"])
df['scenario'] = df.scenario.apply(lambda x: scenarios[x])
df['c'] = 1
df.groupby("scenario").sum().sort_values(by="c", ascending=False)

In [ ]:
df = pd.DataFrame(top_5_intervene_before_sq, columns=["scenario"])
df['scenario'] = df.scenario.apply(lambda x: scenarios[x])
df['c'] = 1
df.groupby("scenario").sum().sort_values(by="c", ascending=False)

In [ ]:
torch.tensor(top_5_intervene).shape

In [ ]:
top_5_intervene_sq = torch.tensor(top_5_intervene).reshape((843* 5, 1)).squeeze().tolist()


In [ ]:
df_interv = pd.DataFrame(top_5_intervene_sq, columns=["scenario"])
df_interv['scenario'] = df_interv.scenario.apply(lambda x: scenarios[x])
df_interv['c'] = 1
df_interv.groupby("scenario").sum().sort_values(by="c", ascending=False)

In [ ]:
p = '/kaggle/working/explanation_intervention_cc/'
top3 = []
for elem in os.listdir(p):
    if ".json" not in elem: continue
    with open(p + elem, 'r') as f:
    # Load the data from the file into a Python dictionary:
        data = json.load(f)
    df = pd.DataFrame(list(data.items()), columns=['Text', 'Probability'])
    df["Probability"] = df["Probability"].apply(lambda x: float(x))
    df = df[df["Text"] != "a picture of cars driving during the day"]
    df = df[df["Text"] != "a picture of cars driving in the dark"]
    top3.extend(df.sort_values("Probability", ascending=False)[0:5].Text.tolist())
df = pd.DataFrame(top3)
df['count'] = 1
df.groupby(0).sum().sort_values("count", ascending=False)

In [ ]:
import torch
import clip
from PIL import Image
from utils import *
import pandas as pd
from tqdm import tqdm
from dataloader_comma import *
import json
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

device = "cuda:0"
model, preprocess = clip.load("ViT-L/14", device=device)
multitask = "distance"
dataset_type = 'train'
dataset = CommaDataset(dataset_type=dataset_type, multitask=multitask, return_full=True)
loader = DataLoader(dataset, batch_size=1, num_workers=5)
save_path = '/kaggle/working/explanation_intervention_cc/'

text = clip.tokenize(scenarios).to(device)
scenarios_tokens = scenarios_tokens.to(device)
with torch.no_grad():
    for j, batch in tqdm(enumerate(loader)):
        image_array, vego, angle, distance, gas, brake, ccenabled, seq_key = batch
        intervention = ccenabled.squeeze()

        #switch_indices = np.where(np.diff(intervention) == 1)[0] + 1
        switch_indices = np.where(np.diff(intervention.int()) == -1)[0] + 1

        intervention = np.zeros_like(intervention.squeeze(), dtype=bool)
        intervention[switch_indices] = True

        if intervention.sum() == 0: 
            continue
        

        images = image_array.squeeze()[intervention.squeeze().tolist()]
        if images.shape[0] == 0: 
            del image_array, vego, angle, distance, gas, brake
            continue

        
        img = images
        s = img.shape#[batch_size, seq_len, h,w,c]
        logits_per_image, logits_per_text = model(img.to(device), scenarios_tokens.to(device))
        probs = logits_per_image.detach()
        probs = logits_per_image.softmax(dim=-1).cpu().detach()
        df = pd.DataFrame([scenarios[i.item()] for i in probs.argmax(dim=-1)])
        #df.to_csv(f'{save_path}/{j}.csv')
        
        for i, img in enumerate(images):
            fig, ax = plt.subplots()
            ax.imshow(img.permute(1,2,0).int())
            fig.suptitle(str(df.iloc[i]))
            name = save_path + f"{i}_{j}" + "_image.png"
            result_dict = {scenarios[k]: str(probs[i][k].item()) for k in range(len(scenarios))}
            with open(name.replace('_image.png', ".json"), 'w') as f:
                json.dump(result_dict, f)
            fig.savefig(name)